# Apply tilt aberration to get off-axis PSF

To check the contrast normalization

In [ ]:
import os
from astropy.io import fits
from matplotlib.colors import TwoSlopeNorm, LogNorm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import main_THD, Asterix_root
from Asterix.utils import get_data_dir, read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed

zerocenter = TwoSlopeNorm(vcenter=0.)

In [ ]:
# Instantiate a THD2 object
your_directory = Asterix_root
your_parameter_file_name = 'thd2_setups/Example_param_file_10jul24.ini'
parameter_file_path = os.path.join(your_directory, your_parameter_file_name)

config = read_parameter_file(parameter_file_path)

data_dir = get_data_dir(config_in=config["Data_dir"])
model_local_dir = os.path.join(data_dir, "Model_local")

# Concatenate into the full testbed optical system
thd2 = main_THD.THD2(config, model_local_dir)
original_dm = thd2.dm3.DM_pushact

dm_ref = thd2.dm3.DM_pushact[518] + thd2.dm3.DM_pushact[519] + thd2.dm3.DM_pushact[780]
dm = np.sum(thd2.dm3.DM_pushact, axis=0)
pup = thd2.entrance_pupil.pup
ls = thd2.corono.lyot_pup.pup

## Create the tip/tilt phase screen

Then save it out so it can be read for the THD2 optical model in Asterix.

In [ ]:
px = thd2.dim_overpad_pupil

xvals = (np.arange(px) / px - 0.5)
yvals = (np.arange(px) / px - 0.5)

xx, yy = np.meshgrid(xvals, yvals)

plt.subplot(1, 2, 1)
plt.imshow(xx, cmap='RdBu')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(yy, cmap='RdBu')
plt.colorbar()

In [ ]:
phase = 100 * xx + 100 * yy

plt.imshow(phase, cmap='RdBu')
plt.colorbar()

In [ ]:
fits.writeto('tiptilt.fits', phase, overwrite=True)
# Move the file to your local model dir

## Apply the tip/tilt phase screen and propagate to focal plane intensity

In [ ]:
SIMUconfig={'set_UPphase_abb': True,
            'set_UPrandom_phase': False,
            'UPopd_rms': 0,
            'UPphase_rhoc': 0,
            'UPphase_slope': -3,
            'UPphase_abb_filename': "tiptilt"}

In [ ]:
# Phase upstream of the coronagraph (entrance pup)
phase_abb_up = thd2.generate_phase_aberr(SIMUconfig,
                                         up_or_down='up',
                                         Model_local_dir=model_local_dir,
                                         silence=True)

In [ ]:
# WF in the testbed entrance pupil
input_wavefront = thd2.EF_from_phase_and_ampl(phase_abb=phase_abb_up)
print(np.sum(input_wavefront))

In [ ]:
plt.imshow(np.angle(input_wavefront)**2)
plt.colorbar()

In [ ]:
initialFP = thd2.todetector_intensity(entrance_EF=input_wavefront, voltage_vector=0, nb_photons=0)
plt.imshow(initialFP, norm=LogNorm(), origin='lower', cmap='inferno')
plt.colorbar()
print(np.max(initialFP))